In [1]:
import numpy as np

num_samples = 217636
x_shape = (100, 200, 3)
y_shape = (2,)

x_dtype = 'float32'  # Determine the appropriate dtype
y_dtype = 'float32'  # Determine the appropriate dtype

# x_memmap = np.memmap('x_dataset.memmap', dtype=x_dtype, mode='w+', shape=(num_samples,) + x_shape)
# y_memmap = np.memmap('y_dataset.memmap', dtype=y_dtype, mode='w+', shape=(num_samples,) + y_shape)


In [2]:
# load x_memmap and y_memmap
x_memmap = np.memmap('x_dataset.memmap', dtype=x_dtype, mode='r+', shape=(num_samples,) + x_shape)
y_memmap = np.memmap('y_dataset.memmap', dtype=y_dtype, mode='r+', shape=(num_samples,) + y_shape)

In [3]:
import pickle
import glob
import numpy as np

def process_and_combine_pkl_files_to_memmap(directory_path, x_memmap, y_memmap):
    current_index = 0
    
    for file_path in glob.glob(directory_path + '/*.pkl'):
        with open(file_path, 'rb') as file:
            data = pickle.load(file)
            
        # Check if data is a dictionary with 'X' and 'Y' keys
        if isinstance(data, dict) and 'X' in data and 'Y' in data:
            X_data = data['X']
            Y_data = data['Y']
        else:
            # Assume that the data is just the raw arrays for X and Y
            Y_numeric = np.array([y[:2] for y in data[1]], dtype=np.float32)  # Convert the first two elements to float
            X_data = np.array(data[0], dtype=np.float32)
            Y_data = Y_numeric

        num_samples_in_file = len(X_data)
        x_batch = np.array(X_data, dtype=x_memmap.dtype).reshape((num_samples_in_file,) + x_shape)
        y_batch = np.array(Y_data, dtype=y_memmap.dtype).reshape((num_samples_in_file,) + y_shape)
        
        # Ensure we do not exceed the allocated memmap size
        if current_index + num_samples_in_file > len(x_memmap):
            raise ValueError("The dataset is larger than expected.")
        
        # Write directly to the memmap files
        x_memmap[current_index:current_index + num_samples_in_file] = x_batch
        y_memmap[current_index:current_index + num_samples_in_file] = y_batch
        
        current_index += num_samples_in_file
        x_memmap.flush()
        y_memmap.flush()


In [3]:
directory_path = './process_MPIIGaze/batches/' 
process_and_combine_pkl_files_to_memmap(directory_path, x_memmap, y_memmap)

In [2]:
def memmap_batch_generator(x_memmap_path, y_memmap_path, batch_size, shuffle=True):
    x_memmap = np.memmap(x_memmap_path, dtype=x_dtype, mode='r', shape=(num_samples,) + x_shape)
    y_memmap = np.memmap(y_memmap_path, dtype=y_dtype, mode='r', shape=(num_samples,) + y_shape)
    
    indices = np.arange(len(x_memmap))
    if shuffle:
        np.random.shuffle(indices)
    
    while True:
        for start_idx in range(0, len(indices), batch_size):
            end_idx = min(start_idx + batch_size, len(indices))
            batch_indices = indices[start_idx:end_idx]
            
            # Ensure batch_indices is an array of integers
            batch_indices = np.array(batch_indices, dtype=np.int32)
            
            # Yield a batch of data
            yield x_memmap[batch_indices], y_memmap[batch_indices]

In [3]:
batch_size = 32

In [4]:

# Paths to your memmap files
x_memmap_path = 'x_dataset.memmap'
y_memmap_path = 'y_dataset.memmap'

# Calculate steps per epoch and validation steps
num_train_samples = int(num_samples * 0.7)  
num_val_samples = int(num_samples * 0.15)   

steps_per_epoch = num_train_samples // batch_size
validation_steps = num_val_samples // batch_size

In [ ]:
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, Flatten, Dropout, BatchNormalization
from keras.models import Model
from keras.optimizers import Adam
from keras.regularizers import l2

# Input layer
input_layer = Input(shape=(100, 200, 3))

# Convolutional layers
x = Conv2D(32, (7, 7), activation='relu', kernel_regularizer=l2(0.001))(input_layer)
x = Conv2D(64, (7, 7), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)
x = BatchNormalization()(x)
x = Dropout(0.15)(x)

x = Conv2D(128, (5, 5), activation='relu', kernel_regularizer=l2(0.001))(x)
x = MaxPooling2D((2, 2))(x)
x = Dropout(0.15)(x)
x = BatchNormalization()(x)

x = Conv2D(256, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)

x = Conv2D(512, (3, 3), activation='relu')(x)
x = Dropout(0.15)(x)

x = Conv2D(1024, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)
x = Dropout(0.175)(x)

x = Flatten()(x)
x = Dense(64, activation='relu')(x)
x = Dropout(0.2)(x)

# Output layer: Adjusted to output 8 values (2 for gaze, 6 for head pose)
output_layer = Dense(8, activation='sigmoid')(x)

# Model definition
model = Model(inputs=input_layer, outputs=output_layer)

model.compile(optimizer=Adam(learning_rate=0.00005), loss='mse', metrics=['mean_squared_error', 'mean_absolute_error'])


In [5]:
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Conv2D, Dense, Dropout, Flatten
from keras.regularizers import l2
from keras.layers import MaxPooling2D
from keras.layers import BatchNormalization

# Your model definition
model = Sequential([
    Conv2D(32, (7, 7), activation='relu', input_shape=(100, 200, 3), kernel_regularizer=l2(0.001)),
    
    
    Conv2D(64, (7, 7), activation='relu'),
    MaxPooling2D((2, 2)),
    BatchNormalization(),
    Dropout(0.15),
 

    Conv2D(128, (5, 5), activation='relu', kernel_regularizer=l2(0.001)),
    MaxPooling2D((2, 2)),
    Dropout(0.15),
    BatchNormalization(),
    

    Conv2D(256, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),

    Conv2D(512, (3, 3), activation='relu'),
    # MaxPooling2D((2, 2)),
    Dropout(0.15),

    Conv2D(1024, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Dropout(0.175),

    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(2, activation='sigmoid')
])


model.compile(optimizer= Adam(learning_rate=0.00005), loss='mse', metrics=['mean_squared_error', 'mean_absolute_error'])

early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Define the checkpoint callback to save every 5 epochs
checkpoint = ModelCheckpoint('eye_gaze_v23_2_{epoch:02d}.h5', save_freq=5*steps_per_epoch)


In [6]:
# Assuming you have already set up your model

# Create generators
train_generator = memmap_batch_generator(x_memmap_path, y_memmap_path, batch_size, shuffle=True)
validation_generator = memmap_batch_generator(x_memmap_path, y_memmap_path, batch_size, shuffle=False)  # Assuming you can use the same for simplicity


# Train the model
model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=100,  # Adjust as needed
    validation_data=validation_generator,
    validation_steps=validation_steps,
    callbacks=[early_stopping, checkpoint]
)

Epoch 1/100
4760/4760 [==============================] - 897s 186ms/step - loss: 0.0507 - mean_squared_error: 0.0256 - mean_absolute_error: 0.1182 - val_loss: 0.0193 - val_mean_squared_error: 0.0117 - val_mean_absolute_error: 0.0758
Epoch 2/100
4760/4760 [==============================] - 706s 148ms/step - loss: 0.0191 - mean_squared_error: 0.0137 - mean_absolute_error: 0.0853 - val_loss: 0.0105 - val_mean_squared_error: 0.0064 - val_mean_absolute_error: 0.0542
Epoch 3/100
4760/4760 [==============================] - 704s 148ms/step - loss: 0.0149 - mean_squared_error: 0.0113 - mean_absolute_error: 0.0768 - val_loss: 0.0087 - val_mean_squared_error: 0.0055 - val_mean_absolute_error: 0.0526
Epoch 4/100
4760/4760 [==============================] - 704s 148ms/step - loss: 0.0129 - mean_squared_error: 0.0101 - mean_absolute_error: 0.0725 - val_loss: 0.0081 - val_mean_squared_error: 0.0055 - val_mean_absolute_error: 0.0552
Epoch 5/100
4760/4760 [==============================] - 705s 148ms/

In [3]:
#load the model
from keras.models import load_model
model = load_model('./models/eye_gaze_v23V99.h5')

In [4]:
# Example: Evaluate on a test set
x_test_memmap = np.memmap('x_dataset.memmap', dtype=x_dtype, mode='r', shape=(num_samples,) + x_shape)
y_test_memmap = np.memmap('y_dataset.memmap', dtype=y_dtype, mode='r', shape=(num_samples,) + y_shape)

# Assuming the last 15% of the data is for testing
test_start_index = int(num_samples * 0.99)
x_test = x_test_memmap[test_start_index:]
y_test = y_test_memmap[test_start_index:]

model.evaluate(x_test, y_test, batch_size=16)


137/137 [==============================] - 7s 13ms/step - loss: 0.0051 - mean_squared_error: 0.0034 - mean_absolute_error: 0.0425


[0.005079503171145916, 0.003443555673584342, 0.04253864660859108]

In [9]:
model.save('./models/eye_gaze_v23v99.h5')